In [3]:
#PACKAGES PYTHON

import matplotlib.pyplot as plt
import folium
import pandas as pd
from folium import plugins
import pandas as pd
import json
from folium.plugins import TimestampedGeoJson
from geopy.distance import geodesic
import plotly.express as px
import geopandas as gpd
from shapely.geometry import Point
import numpy as np

import pandas as pd
import numpy as np
from geopy.distance import geodesic

In [5]:
import pandas as pd
import tkinter as tk
from tkinter import filedialog

df = None  # Déclaration de la variable globale

def charger_csv():
    global df  # Indique qu'on utilise la variable globale
    filepath = filedialog.askopenfilename(
        title="Choisir un fichier CSV",
        filetypes=[("Fichiers CSV", "*.csv")],
    )
    if filepath:
        try:
            df = pd.read_csv(filepath, sep=';')
            print("CSV chargé avec succès.")
            print("Dimensions du DataFrame :", df.shape)
        except Exception as e:
            print("Erreur lors du chargement du fichier :", e)

# Interface de base avec Tkinter
root = tk.Tk()
root.title("Lecteur CSV")
root.geometry("300x100")

bouton_charger = tk.Button(root, text="Charger un CSV", command=charger_csv)
bouton_charger.pack(pady=20)

root.mainloop()


CSV chargé avec succès.
Dimensions du DataFrame : (625500, 7)
CSV chargé avec succès.
Dimensions du DataFrame : (625500, 7)


In [7]:


def clean_json_str(s):
    if s.startswith('{drone":'):
        s = '{"' + s[1:]
    if s.endswith('}}"'):
        s = s[:-1]
    return s

def extract_values(s):
    try:
        s_clean = clean_json_str(s)
        d = json.loads(s_clean)
        drone = d.get('drone', {})
        altitude = drone.get('altitude', None)
        hauteur = drone.get('hauteur', None)
        vitesse = drone.get('vitesse', None)
        return altitude, hauteur, vitesse
    except Exception as e:
        print(f"Erreur extraction valeurs : {e}")
        return None, None, None

# Appliquer sur toute la colonne properties
df[['altitude', 'hauteur', 'vitesse']] = df['properties'].apply(
    lambda s: pd.Series(extract_values(s))
)



In [8]:
# Supprimer la colonne 'properties' du DataFrame original
df_initial_sans_properties = df

# Faire la concaténation côte à côte
df_drones = pd.concat([df_initial_sans_properties], axis=1)



In [9]:


colonnes_voulues = [
    'drone_id',
    'constructeur',
    'modele',
    'updated',  # temps
    'y',
    'x',
    'altitude',
    'hauteur',
    'vitesse',
    'properties'
]

df_kepler_ready = df_drones[colonnes_voulues].copy()

df_kepler_ready.to_csv('drones_kepler.csv', index=False)



In [10]:
df['updated'] = pd.to_datetime(df['updated'], utc=True, errors='coerce')
print(df['updated'].dtypes)  # Doit afficher le type de date




datetime64[ns, UTC]


In [11]:
import matplotlib.pyplot as plt

# Assure-toi que 'updated' est datetime (avec fuseau horaire ou en UTC)
df['updated'] = pd.to_datetime(df['updated'], utc=True, errors='coerce')

# Nettoyer la colonne 'vitesse'
df['vitesse'] = pd.to_numeric(df['vitesse'], errors='coerce').fillna(0)
# Trier les données pour plus de clarté
df = df.sort_values(['drone_id', 'updated'])

# Liste des drones uniques
drone_ids = df['drone_id'].unique()






In [12]:
# Lire le fichier CSV
df_clean = pd.read_csv('drones_kepler.csv')
# Vérification rapide

In [13]:
# S'assurer que 'updated' est bien en datetime
df_clean['updated'] = pd.to_datetime(df_clean['updated'])

# Trier les données par drone et timestamp
df_clean = df_clean.sort_values(by=['drone_id', 'updated']).reset_index(drop=True)

# Initialiser la colonne des vitesses calculées
df_clean['vitesse_calculee'] = 0.0

# Grouper par drone_id pour calculer la vitesse entre deux points consécutifs
for drone_id, group in df_clean.groupby('drone_id'):
    idxs = group.index
    for i in range(1, len(group)):
        idx_prev = idxs[i - 1]
        idx_curr = idxs[i]

        # Extraire les coordonnées (lat, lon)
        coord_prev = (df_clean.loc[idx_prev, 'y'], df_clean.loc[idx_prev, 'x'])
        coord_curr = (df_clean.loc[idx_curr, 'y'], df_clean.loc[idx_curr, 'x'])

        # Calcul de la distance (mètres)
        distance = geodesic(coord_prev, coord_curr).meters

        # Calcul du temps (secondes)
        t1 = df_clean.loc[idx_prev, 'updated']
        t2 = df_clean.loc[idx_curr, 'updated']
        delta_t = (t2 - t1).total_seconds()

        # Calcul de la vitesse (m/s)
        vitesse = distance / delta_t if delta_t > 0 else 0.0

        df_clean.loc[idx_curr, 'vitesse_calculee'] = vitesse

# Nettoyer la colonne 'vitesse' (s'assurer qu'elle est numérique)
df_clean['vitesse'] = pd.to_numeric(df_clean['vitesse'], errors='coerce')

# Remplacer les valeurs manquantes (NaN) dans 'vitesse' par 'vitesse_calculee'
df_clean['vitesse'] = df_clean['vitesse'].fillna(df_clean['vitesse_calculee'])


C:\Users\fgrol\AppData\Local\Temp\ipykernel_18864\3456638988.py:2: FutureWarning: In a future version of pandas, parsing datetimes with mixed time zones will raise an error unless `utc=True`. Please specify `utc=True` to opt in to the new behaviour and silence this warning. To create a `Series` with mixed offsets and `object` dtype, please use `apply` and `datetime.datetime.strptime`
  df_clean['updated'] = pd.to_datetime(df_clean['updated'])


In [14]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic

# 1. Convertir 'updated' en datetime avec UTC
df_clean['updated'] = pd.to_datetime(df_clean['updated'], utc=True)

# 2. Trier par drone_id et timestamp
df_clean = df_clean.sort_values(by=['drone_id', 'updated']).reset_index(drop=True)

# 3. Calcul du temps écoulé (delta_t) en secondes
df_clean['delta_t'] = df_clean.groupby('drone_id')['updated'].diff().dt.total_seconds()

# 4. Calcul de la vitesse verticale vz (altitude / temps)
df_clean['vz'] = df_clean.groupby('drone_id')['hauteur'].diff() / df_clean['delta_t']

# 5. Calcul de la distance entre points GPS successifs (avec x = lon, y = lat)
def compute_distance(row):
    i = row.name
    if i == 0 or df_clean.loc[i, 'drone_id'] != df_clean.loc[i - 1, 'drone_id']:
        return 0
    coord1 = (df_clean.loc[i - 1, 'y'], df_clean.loc[i - 1, 'x'])  # (lat, lon)
    coord2 = (df_clean.loc[i, 'y'], df_clean.loc[i, 'x'])          # (lat, lon)
    return geodesic(coord1, coord2).m

df_clean['distance'] = df_clean.apply(compute_distance, axis=1)

# 6. Détection de nouvelle trajectoire si delta_t > 5 minutes ou changement de drone
df_clean['new_traj'] = (
    (df_clean['delta_t'] > 300) |
    (df_clean['drone_id'] != df_clean['drone_id'].shift())
).astype(int)

# 7. Attribution d’un ID de trajectoire unique
df_clean['traj_id'] = df_clean.groupby('drone_id')['new_traj'].cumsum()

# 8. Calcul du temps total et distance totale pour chaque trajectoire
traj_stats = df_clean.groupby(['drone_id', 'traj_id']).agg(
    temps_total=('delta_t', 'sum'),
    distance_totale=('distance', 'sum')
).reset_index()

# Nettoyer les colonnes potentielles en doublon avant merge
for col in ['temps_total', 'distance_totale', 'temps_total_x', 'temps_total_y', 'distance_totale_x', 'distance_totale_y']:
    if col in df_clean.columns:
        df_clean.drop(columns=[col], inplace=True)

# 9. Fusion des statistiques dans le dataframe principal
df_clean = df_clean.merge(traj_stats, on=['drone_id', 'traj_id'], how='left')

# 10. Nettoyage de la colonne temporaire
df_clean.drop(columns=['new_traj'], inplace=True)



In [15]:
import matplotlib.pyplot as plt

# Assure que 'vz' est numérique
df_clean['vz'] = pd.to_numeric(df_clean['vz'], errors='coerce').fillna(0)

# Tri par drone, trajectoire et temps
df_clean = df_clean.sort_values(['drone_id', 'traj_id', 'updated'])

# Récupère tous les groupes uniques
group_keys = list(df_clean.groupby(['drone_id', 'traj_id']).groups.keys())

# Vérifie qu'il y a au moins 9 groupes
if len(group_keys) >= 9:
    # On récupère la 9ᵉ (index 8 car indexation commence à 0)
    selected_key = group_keys[8]
    drone, traj = selected_key

    df_sub = df_clean[(df_clean['drone_id'] == drone) & (df_clean['traj_id'] == traj)]


In [16]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd

# Assurer que 'updated' est bien au format datetime
df_clean['updated'] = pd.to_datetime(df_clean['updated'])

# Obtenir les 10 premières combinaisons uniques
top_10_traj = df_clean[['drone_id', 'traj_id']].drop_duplicates().head(10)

# Vérifier qu'on a au moins 9 trajectoires
if len(top_10_traj) >= 9:
    # Récupérer la 9ᵉ trajectoire (index 8)
    row = top_10_traj.iloc[8]
    drone_id = row['drone_id']
    traj_id = row['traj_id']

    # Filtrer les données de cette trajectoire
    group = df_clean[(df_clean['drone_id'] == drone_id) & (df_clean['traj_id'] == traj_id)]
    group = group.sort_values('updated')

    # Calcul de la distance totale
    distance_totale = group['distance'].sum()

In [17]:

# Fonction pour calculer la distance max au point (x0, y0)
def compute_max_distance_to_start(group):
    x0, y0 = group.iloc[0][['x', 'y']]  # point de la télécommande
    distances = np.sqrt((group['x'] - x0)**2 + (group['y'] - y0)**2)
    max_distance = distances.max()
    group['distance_max_telecommande'] = max_distance
    return group

# Appliquer la fonction par trajectoire
df_clean = df_clean.groupby(['drone_id', 'traj_id'], group_keys=False).apply(compute_max_distance_to_start)





C:\Users\fgrol\AppData\Local\Temp\ipykernel_18864\3502637546.py:10: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_clean = df_clean.groupby(['drone_id', 'traj_id'], group_keys=False).apply(compute_max_distance_to_start)


In [18]:


def clean_json_str(s):
    if s.startswith('{drone":'):
        s = '{"' + s[1:]
    if s.endswith('}}"'):
        s = s[:-1]
    return s

def extract_lat_lon(s):
    try:
        s_clean = clean_json_str(s)
        d = json.loads(s_clean)
        drone = d.get('drone', {})
        latitude_depart = drone.get('latitude_depart', None)
        longitude_depart = drone.get('longitude_depart', None)
        return latitude_depart, longitude_depart
    except Exception as e:
        print(f"Erreur extraction coordonnées : {e}")
        return None, None

# Application sur df_clean
df_clean[['latitude_depart', 'longitude_depart']] = df_clean['properties'].apply(
    lambda s: pd.Series(extract_lat_lon(s))
)


In [19]:
import numpy as np

# Définir les bornes valides de la France métropolitaine
lat_min, lat_max = 41.0, 51.5
lon_min, lon_max = -5.5, 9.5

# Masques de validité
lat_valid = df_clean['latitude_depart'].between(lat_min, lat_max)
lon_valid = df_clean['longitude_depart'].between(lon_min, lon_max)

# Remplacer par NaN si hors des bornes
df_clean.loc[~lat_valid, 'latitude_depart'] = np.nan
df_clean.loc[~lon_valid, 'longitude_depart'] = np.nan


In [20]:
import numpy as np

def haversine(lat1, lon1, lat2, lon2):
    R = 6371000  # rayon terrestre en mètres
    phi1, phi2 = np.radians(lat1), np.radians(lat2)
    dphi = np.radians(lat2 - lat1)
    dlambda = np.radians(lon2 - lon1)

    a = np.sin(dphi/2)**2 + np.cos(phi1) * np.cos(phi2) * np.sin(dlambda/2)**2
    c = 2 * np.arctan2(np.sqrt(a), np.sqrt(1 - a))

    return R * c  # distance en mètres

# Distance par rapport au point de départ GPS
df_clean['distance_from_depart'] = df_clean.apply(
    lambda row: haversine(row['latitude_depart'], row['longitude_depart'], row['y'], row['x']), axis=1
)

# Distance maximale par groupe
df_clean['distance_depart'] = df_clean.groupby(['drone_id', 'traj_id'])['distance_from_depart'].transform('max')
# Remplacer la colonne par une version formatée en string
df_clean['distance_depart'] = df_clean['distance_depart'].apply(lambda x: f"{x:.2f}")

df_clean.drop(columns=['distance_from_depart'], inplace=True)



In [21]:
import pandas as pd
import unidecode
from rapidfuzz import process

# === 1. Liste officielle des modèles ===
modeles_ref = [
    "Mini 4K", "Mini 3 Pro", "Mini 3", "Mini 2 SE", "Mini SE", "Air 3S", "Air 3", "Air 2S",
    "Mavic 4 Pro", "Mavic 3 Pro", "Mavic 3 Classic", "Mavic 3", "Mavic 2", "Mavic Pro",
    "Mavic Pro Platinum", "Avata", "FPV (original)", "Inspire 3", "Inspire 2",
    "Matrice 300 RTK", "Matrice 350 RTK", "Matrice 30", "Matrice 30T", "Matrice 400",
    "Matrice 4E", "Matrice 4T", "Mavic 3T", "Mavic 3E", "Mavic 3M",
    "Mavic 2 Enterprise Dual", "Mavic 2 Enterprise Advanced", "Agras T50", "Agras T25",
    "T20", "T25P", "T30", "T60", "T100", "MG-1", "Phantom 4 Pro", "Phantom 4 Advanced",
    "Spreading Wings S1000", "Evo Lite+", "Evo Nano+", "Typhoon H", "Typhoon H Plus",
    "Breeze", "Ghost Drone 2.0", "Atom 2", "Anafi", "Anafi USA", "Mavic 2 Zoom",
    "Phantom 4 RTK", "Extra 330S"
]

# === 2. Fonction de nettoyage des chaînes ===
def normalize(s):
    if pd.isna(s):
        return ""
    s = str(s)
    s = unidecode.unidecode(s).lower()
    s = s.replace("-", " ").replace("_", " ").strip()
    return s

# === 3. Normaliser la liste de référence ===
normalized_ref = {normalize(m): m for m in modeles_ref}
normalized_ref_list = list(normalized_ref.keys())

# === 4. Fonction de correspondance floue robuste ===
def match_model(s, threshold=85):
    s_norm = normalize(s)
    result = process.extractOne(s_norm, normalized_ref_list, score_cutoff=threshold)
    if result:
        match, score, _ = result
        return normalized_ref[match]
    return None

# === 5. Optimisation : matching sur valeurs uniques ===
valeurs_uniques = df_clean["modele"].dropna().unique()
correspondance = {val: match_model(val) for val in valeurs_uniques}

# === 6. Remplacement dans le DataFrame ===
df_clean["modele_standardise"] = df_clean["modele"].map(correspondance)

# === 7. (Optionnel) Export des modèles non reconnus ===
non_reconnus = {k: v for k, v in correspondance.items() if v is None}
pd.Series(non_reconnus).to_csv("modeles_non_reconnus.csv")

df_clean.to_csv("csv_test.csv", index=False)


In [22]:
modeles_uniques = df_clean["modele_standardise"].dropna().unique()
modeles_uniques.sort()
for i, modele in enumerate(modeles_uniques):
    print(f"{i+1}: {modele}")




1: Air 2S
2: Air 3
3: Air 3S
4: Avata
5: Extra 330S
6: Inspire 3
7: Matrice 300 RTK
8: Matrice 30T
9: Matrice 350 RTK
10: Matrice 4E
11: Matrice 4T
12: Mavic 2
13: Mavic 2 Enterprise Advanced
14: Mavic 2 Enterprise Dual
15: Mavic 2 Zoom
16: Mavic 3
17: Mavic 3 Classic
18: Mavic 3 Pro
19: Mavic 3E
20: Mavic 3M
21: Mavic 3T
22: Mavic 4 Pro
23: Mavic Pro
24: Mini 2 SE
25: Mini 3 Pro
26: Mini 4K
27: Mini SE
28: Phantom 4 Pro
29: Phantom 4 RTK
